In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import json
import jsonlines

In [2]:
NER = ["EFFECT"]
set = "test"

In [3]:
l = []
for ner in NER:
	test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
	test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
	merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
	merged.loc[:,"ner"] = ner
	l.append(merged)

df = pd.concat(l)

In [4]:
df = df.reset_index().drop(columns=["index"])

In [5]:
df

,tokens,ner_tags,id,predictions,label_ids,word_ids,ner
0,"[The, strain, N., spumigena, CCY9414, was, ini...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[[-2.808884143829345, -2.834216117858886, 5.92...","[-100, 2, 2, 2, -100, 2, -100, -100, -100, 2, ...","[None, 0, 1, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, ...",EFFECT
1,"[The, PCR, assay, designed, to, target, yjcS, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1,"[[-3.221582651138305, -2.591535329818725, 5.54...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 9, 10,...",EFFECT
2,"[Biosurfactant, production, was, carried, out,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2,"[[-3.277575016021728, -2.303949832916259, 5.74...","[-100, 2, -100, -100, -100, 2, 2, 2, 2, 2, 2, ...","[None, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, ...",EFFECT
3,"[We, started, to, investigate, the, growth, fa...","[O, O, O, O, O, B, I, I, O, O, O, O, O, O, O, ...",3,"[[-2.385529279708862, -2.147912502288818, 4.70...","[-100, 2, 2, 2, 2, 2, 0, 1, -100, -100, 1, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 9, 9, ...",EFFECT
4,"[To, address, this, gap, in, our, knowledge, o...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",4,"[[-2.314257621765136, -1.9107927083969112, 4.9...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, 2, -10...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 9, 9, 10,...",EFFECT
...,...,...,...,...,...,...,...
619,"[The, LH1, subunits, of, strain, 970, are, uni...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",619,"[[-3.138967514038086, -2.332874298095703, 5.41...","[-100, 2, 2, -100, 2, 2, 2, 2, -100, 2, 2, 2, ...","[None, 0, 1, 1, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10,...",EFFECT
620,"[Genomic, DNA, was, isolated, from, yeast, cul...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",620,"[[-2.612555742263794, -2.514002323150634, 5.33...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 11...",EFFECT
621,"[Syringaldehyde, and, sinapaldehyde, ,, for, e...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",621,"[[-2.571874856948852, -2.437544107437134, 4.82...","[-100, 2, -100, 2, 2, -100, -100, -100, -100, ...","[None, 0, 0, 1, 2, 2, 2, 2, 2, 2, 3, 4, 5, 6, ...",EFFECT
622,"[This, study, is, based, on, cases, that, were...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",622,"[[-3.01359748840332, -2.418065071105957, 5.210...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",EFFECT


In [6]:
label_ids = df[["label_ids"]]

In [7]:
annotations = []
for j in range(len(df)):
	for i in zip(df.loc[j,"tokens"],df.loc[j,"ner_tags"]):
		if i[1] != "O":
			annotations.append({"label":i[1],"text":i[0]})

### Predict sentences again

In [8]:
ner = "EFFECT"

In [9]:
path= f"../NER_output/{ner}"

tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
model = AutoModelForTokenClassification.from_pretrained(path)

nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
               aggregation_strategy="average", device=0)


In [10]:
# Specify the path to the JSONL file
jsonl_file = f"../NER/{ner}/test.jsonls"

predictions = []
# Open the JSONL file
with jsonlines.open(jsonl_file) as reader:
    # Iterate over each line in the file
    for line in reader:
        for l in line:
            # Get the value from the "text" field
            text = l["data"]["text"]
            # Pass the value to the nlp() function
            print(text)
            predict = nlp(text)
            predictions.append(predict)
            print(predict)

The strain N. spumigena CCY9414 was initially isolated from samples collected from surface water in the Bornholm Sea ( Hayes and Barker, 1997 ; Stal et al., 2003 ; Voss et al., 2013 ).
[]
The PCR assay designed to target yjcS , which encodes a metallo-β-hydrolase in the A. dhakensis genome, yielded the predicted 223-bp amplicon at 62°C only for the DNA samples derived from the two A. dhakensis isolates collected from striped catfish in this present study (TN5 and TN49) and the two A. dhakensis type strains (CIP 107500 ^T and CECT 7289 ^T ).
[]
Biosurfactant production was carried out by culturing R . opacus R7 in a 3 L flask containing 500 mL M9 mineral medium [ 31 ] supplemented with naphthalene (1 g naphthalene crystals/L of culture) as the only carbon and energy source at 30 °C under shaking (120 rpm) up to 96 h corresponding to an optical density at 600 nm (O.D. _600 ) of around 2.
[]
We started to investigate the growth factor-like activity of TpF1, a major antigen expressed by T.

/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[]
The resulting DQ12-45-1b streptomycin-resistant strain was cultured in GPY-containing streptomycin.
[]
The findings indicated that cell-free culture supernatant derived from L. reuteri AN417 exhibited the highest activity against P. gingivalis and potentially against other oral pathogenic bacteria.
[]
It was also determined that the E. sp. nr. emiratus hosts of E. suzannae males contain Wolbachia [ 36 ] ; thus, the Wolbachia wPip genome was added and mapped to the male samples.
[]
Instead of forming fibrils, which are insoluble, high-molecular-weight species, A30P forms relatively soluble, partially aggregated species (Conway et al.
[]
A significant difference is the higher presence of C _15:1 ω 6c detected in the P4023 ^T triad that enables its clear separation from the compared strains.
[]
The 77 kbp region is a hotspot for SNPs in both the Australian koala strains ( C. pecorum MC/Marsbar, C. pecorum IPTaLE and C. pecorum DBDeUG) and European livestock strains ( C. pecorum PV3056,

In [11]:
predictions = [p for p in predictions if p]

In [12]:
predictions

[[{'entity_group': 'I',
   'score': 0.537879,
   'word': 'receptor processing',
   'start': 64,
   'end': 83}],
 [{'entity_group': 'I',
   'score': 0.48725668,
   'word': 'diarrheal foodborne illness',
   'start': 223,
   'end': 250}],
 [{'entity_group': 'I',
   'score': 0.5983044,
   'word': 'the activation of dcs and systemic antitumor effect',
   'start': 150,
   'end': 201}],
 [{'entity_group': 'I',
   'score': 0.39085287,
   'word': 'proliferation',
   'start': 38,
   'end': 51}],
 [{'entity_group': 'I',
   'score': 0.5888595,
   'word': 'host specialization',
   'start': 44,
   'end': 63}],
 [{'entity_group': 'I',
   'score': 0.41503346,
   'word': 'morphological',
   'start': 135,
   'end': 148},
  {'entity_group': 'I',
   'score': 0.490148,
   'word': 'biochemical improvement of plants',
   'start': 153,
   'end': 186},
  {'entity_group': 'I',
   'score': 0.48297888,
   'word': 'root and shoot length',
   'start': 194,
   'end': 215},
  {'entity_group': 'I',
   'score': 0.54002

In [13]:
annotations

[{'label': 'B', 'text': 'growth'},
 {'label': 'I', 'text': 'factor-like'},
 {'label': 'I', 'text': 'activity'},
 {'label': 'B', 'text': 'proliferative'},
 {'label': 'I', 'text': 'rate'},
 {'label': 'B', 'text': 'rescue'},
 {'label': 'I', 'text': 'receptor'},
 {'label': 'I', 'text': 'processing'},
 {'label': 'I', 'text': 'in'},
 {'label': 'I', 'text': 'LoVo'},
 {'label': 'I', 'text': 'cells'},
 {'label': 'B', 'text': 'adaptive'},
 {'label': 'I', 'text': 'responses'},
 {'label': 'I', 'text': 'of'},
 {'label': 'I', 'text': 'strain'},
 {'label': 'I', 'text': 'S12'},
 {'label': 'B', 'text': 'Fe^0-assisted'},
 {'label': 'I', 'text': 'azo'},
 {'label': 'I', 'text': 'reduction'},
 {'label': 'I', 'text': 'processes'},
 {'label': 'B', 'text': 'irradiation'},
 {'label': 'B', 'text': 'antitumor'},
 {'label': 'B', 'text': 'elevated'},
 {'label': 'I', 'text': 'expression'},
 {'label': 'I', 'text': 'of'},
 {'label': 'I', 'text': 'PPARα'},
 {'label': 'B', 'text': 'cell'},
 {'label': 'I', 'text': 'prol

In [14]:
for prediction in predictions:
    pred = prediction[0]["word"]
    count = 0
    for annotation in annotations:
        ann = annotation["text"]
        if ann in pred:
            count +=1
    if count <2:
        print(pred)

sensitivity
delivery
sensitivity
teer


In [15]:
ners =["COMPOUND", "DISEASE", "EFFECT","ISOLATE","MEDIUM","METABOLITE","ORGANISM",
            "PHENOTYPE","PROTEIN","SPECIES","STRAIN"]
for ner in ners:
    test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
    test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
    merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
    merged.loc[:,"ner"] = ner
    label_ids = merged[["label_ids"]]

    annotations = []
    for j in range(len(merged)):
    	for i in zip(merged.loc[j,"tokens"],merged.loc[j,"ner_tags"]):
    		if i[1] != "O":
    			annotations.append({"label":i[1],"text":i[0]})
    
    path= f"../NER_output/{ner}"
    
    tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
    model = AutoModelForTokenClassification.from_pretrained(path)
    
    nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
                   aggregation_strategy="average", device=0)

    # Specify the path to the JSONL file
    jsonl_file = f"../NER/{ner}/test.jsonls"
    
    predictions = []
    # Open the JSONL file
    with jsonlines.open(jsonl_file) as reader:
        # Iterate over each line in the file
        for line in reader:
            for l in line:
    
                # Get the value from the "text" field
                text = l["data"]["text"]
                # Pass the value to the nlp() function
                predict = nlp(text)
                predictions.append(predict)
    predictions = [p for p in predictions if p]
    print("-----")
    print(ner)
    print("-----")
    for prediction in predictions:
        pred = prediction[0]["word"]
        pred.replace(" - ","-")
        count = 0
        for annotation in annotations:
            ann = annotation["text"].lower()
            if ann in pred:
                count +=1
        if count <1:
            print(pred)

/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-----
COMPOUND
-----
bms
nh
ht115
n
sulfide
rifamycin
ga
4
al
iva
80
fe
_
mk
mn
trans
dextran
pi


/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-----
DISEASE
-----
gastric
hyperlipidemia
melioidosis
clubroot
covid
sclc
pediatric


/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

-----
PROTEIN
-----
gp2
di
pod
efflux pump
pump
γ
nt
opn4
α
e
ifn
g6pdh
il
α
3
β
lrd
2
n
β
perk
spike
l


/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-----
SPECIES
-----
h
l
p
b
a
l
l
h
b
v
p
x
lb
b
l
b
r
p
a
a
p
b
a
p
n
p
b
h
n
p
p
b
p


/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-----
STRAIN
-----
l
w
s
dq12
l
r
s3
n
1980
l
s
g
s
x
s
dq12
w
g
‘
r
81
s
l
95
dw84
l
n2
n
s17
dq12
s
